In [1]:
import pandas as pd
from neo4j import GraphDatabase


# Load the processed data
# Replace 'processed_data.csv' with the actual path to your processed data file
data = pd.read_csv('../data/processed/processed_data_with_embeddings.csv')

# Display the first few rows of the data to verify
data.head()

,Image_ID,Document,Named_Entities,Embeddings
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,View towards South East of junction of Queen V...,"[{'ne_span': 'Queen Victoria Road', 'ne_start'...","[0.04749252647161484, -0.01967967115342617, -0..."
1,c29a7b77-7c46-3b85-88fe-05c8f4b2e384,"Front page of Bucks Free Press, Time capsule f...","[{'ne_span': 'Bucks Free Press', 'ne_start': 1...","[-0.07886386662721634, 0.008736064657568932, 0..."
2,196c11e6-f7b6-392f-ae41-28653345087c,"High Wycombe Police Station, in Queen Victoria...","[{'ne_span': 'High Wycombe Police Station', 'n...","[-0.0424775592982769, -0.0020039905793964863, ..."
3,7a5aace6-2398-3dcf-8843-37ff6ccea875,"Reference Library door, Queen Victoria Road , ...","[{'ne_span': 'Reference Library', 'ne_start': ...","[0.008828028105199337, 0.003003709716722369, -..."
4,c66c4715-c03a-3aab-964b-e733f3ff1cf4,"Terrace of brick and flint cottages, Beech Roa...","[{'ne_span': 'Beech Road ', 'ne_start': 37, 'n...","[0.030101340264081955, 0.028815921396017075, -..."


In [ ]:
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "12345678"

def run_query(query):
    """Execute a Cypher query and return the result."""
    with GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD)) as driver:
        with driver.session() as session:
            result = session.run(query)
            return result.data()
        



In [ ]:
query_normalise_wiki_id = """
    MATCH (e:NamedEntity)
    WHERE e.wiki_id STARTS WITH "https://www.wikidata.org/wiki/"
    SET e.wikiId = replace(e.wiki_id, "https://www.wikidata.org/wiki/", "")
    REMOVE e.wiki_id
    RETURN e.text, e.wikiId
    LIMIT 10;
    """

run_query(query_normalise_wiki_id)

[{'e.text': 'Bucks Free Press', 'e.wikiId': 'Q4983472'},
 {'e.text': 'Clock House', 'e.wikiId': 'Q63302141'},
 {'e.text': 'Reference Library', 'e.wikiId': 'Q7075'},
 {'e.text': 'Reference', 'e.wikiId': 'Q121769'},
 {'e.text': 'Wycombe Marsh', 'e.wikiId': 'Q15054211'},
 {'e.text': "Working Men's Club", 'e.wikiId': 'Q8034980'},
 {'e.text': 'Library', 'e.wikiId': 'Q7075'},
 {'e.text': 'Post Office', 'e.wikiId': 'Q1783168'},
 {'e.text': 'White Hart', 'e.wikiId': 'Q7994771'},
 {'e.text': 'Guildhall', 'e.wikiId': 'Q81279'}]

In [13]:
# Link Named Entities to a Wikidata Concept
query_link_wikidata = """
    MATCH (e:NamedEntity)
    WHERE e.wikiId IS NOT NULL
    MERGE (w:WikidataConcept {wikiId: e.wikiId})
    ON CREATE SET w.label = e.text
    MERGE (e)-[:LINKS_TO]->(w);
    """

run_query(query_link_wikidata)

[]

In [3]:
from SPARQLWrapper import SPARQLWrapper, JSON
from neo4j import GraphDatabase

# Connect to Neo4j
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "12345678"
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Function to fetch descriptions from Wikidata
def get_wikidata_description(wiki_id):
    query = f"""
    SELECT ?description ?instanceOfLabel WHERE {{
      wd:{wiki_id} schema:description ?description.
      OPTIONAL {{ wd:{wiki_id} wdt:P31 ?instanceOf. ?instanceOf rdfs:label ?instanceOfLabel. FILTER (lang(?instanceOfLabel) = "en") }}
      FILTER (lang(?description) = "en")
    }} LIMIT 1
    """
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    if results["results"]["bindings"]:
        result = results["results"]["bindings"][0]
        description = result.get("description", {}).get("value", "No description available")
        instance_of = result.get("instanceOfLabel", {}).get("value", "Unknown type")
        return description, instance_of
    return "No description available", "Unknown type"

# Update Neo4j with fetched data
def update_neo4j():
    with driver.session() as session:
        wikidata_nodes = session.run("MATCH (w:WikidataConcept) RETURN w.wikiId AS wikiId")
        for record in wikidata_nodes:
            wiki_id = record["wikiId"]
            description, instance_of = get_wikidata_description(wiki_id)
            session.run(
                "MATCH (w:WikidataConcept {wikiId: $wikiId}) "
                "SET w.description = $description, w.type = $instanceOf",
                wikiId=wiki_id, description=description, instanceOf=instance_of
            )

update_neo4j()


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)>

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

# Disable SSL verification
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Function to fetch descriptions from Wikidata
def get_wikidata_description(wiki_id):
    query = f"""
    SELECT ?description ?instanceOfLabel WHERE {{
      wd:{wiki_id} schema:description ?description.
      OPTIONAL {{ wd:{wiki_id} wdt:P31 ?instanceOf. ?instanceOf rdfs:label ?instanceOfLabel. FILTER (lang(?instanceOfLabel) = "en") }}
      FILTER (lang(?description) = "en")
    }} LIMIT 1
    """
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    # Disable SSL verification
    sparql._defaultRequest = lambda: sparql._createRequest(ssl._create_unverified_context())

    results = sparql.query().convert()
    if results["results"]["bindings"]:
        result = results["results"]["bindings"][0]
        description = result.get("description", {}).get("value", "No description available")
        instance_of = result.get("instanceOfLabel", {}).get("value", "Unknown type")
        return description, instance_of
    return "No description available", "Unknown type"


# Update Neo4j with fetched data
def update_neo4j():
    with driver.session() as session:
        wikidata_nodes = session.run("MATCH (w:WikidataConcept) RETURN w.wikiId AS wikiId")
        for record in wikidata_nodes:
            wiki_id = record["wikiId"]
            description, instance_of = get_wikidata_description(wiki_id)
            session.run(
                "MATCH (w:WikidataConcept {wikiId: $wikiId}) "
                "SET w.description = $description, w.type = $instanceOf",
                wikiId=wiki_id, description=description, instanceOf=instance_of
            )

update_neo4j()


NameError: name 'driver' is not defined